In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import matplotlib as mpl
import torch
import random
import os
import json
from tqdm import tqdm
import joblib
from joblib import Parallel

In [2]:
with open('./japanese_characters.txt', 'r') as f:
  chars = f.read()

kanji_chars = chars[92:]

font_dirs = ['../all-fonts', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)
font_list = fm.createFontList(font_files)
fm.fontManager.ttflist.extend(font_list)

/var/folders/8h/gq3wkjn971g716fqf8xqlpch0000gn/T/ipykernel_80194/1971218514.py:8: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = fm.createFontList(font_files)


In [3]:
# style_font_path =  '../fonts/JK-Maru-Gothic-M.otf'
# style_name = os.path.splitext(os.path.basename(style_font_path))[0]
# with open(f'../statistic/loss_dictionary/{style_name}_kanji.json', 'r') as f:
#   all_loss_info = json.load(f)
# columns = list(all_loss_info.keys())
# index = list(kanji_chars)
# data = np.zeros((len(index), len(columns)))
# for i in range(len(columns)):
#   tmp_column = [info['L1'] for info in list(all_loss_info[columns[i]].values())]
#   data[:, i] = tmp_column
# df = pd.DataFrame(data, index=index, columns=columns)
# df

In [3]:
# result_info = {}
# num = 4
# top = 100
# bottom = 100
# trial = 100000
# indexes = [0] * num
# marked_set = set()
# for i in tqdm(range(trial)):
#   while True:
#     for i in range(num):
#       indexes[i] = random.randint(0, len(index) - 1)
#     sorted_indexed = sorted(indexes)
#     if tuple(sorted_indexed) not in marked_set:
#       marked_set.add(tuple(sorted_indexed))
#       break
#   result = {}
#   chars = ''
#   for i in range(num):
#     chars += index[sorted_indexed[i]]
#   tmp_data = data[:, sorted_indexed]
#   loss_amount = tmp_data.min(axis=1).sum()
#   loss_min_index = tmp_data.argmin(axis=1, )

#   result['index'] = indexes
#   result['loss_amount'] = loss_amount
#   result['loss_min_index'] = [int(l) for l in loss_min_index]
#   result_info[chars] = result

# sorted_result_info = {
#   key: value for key, value in sorted(result_info.items(), key=lambda item: item[1]['loss_amount'])
# }
# top_bottom_result_info = {}
# top_keys = list(sorted_result_info.keys())[:top]
# bottom_keys = list(sorted_result_info.keys())[-bottom:]
# top_values = list(sorted_result_info.values())[:top]
# bottom_values = list(sorted_result_info.values())[-bottom:]
# for key, value in zip(top_keys, top_values):
#     top_bottom_result_info[key] = value
# for key, value in zip(bottom_keys, bottom_values):
#     top_bottom_result_info[key] = value

# result_dir_path = f'../statistic/loss_dictionary/{style_name}'
# result_file_path = os.path.join(result_dir_path, f'{trial}.json')
# if not os.path.exists(result_dir_path):
#   os.makedirs(result_dir_path)
# with open(result_file_path, 'w') as f:
#   json.dump(top_bottom_result_info, f)

In [4]:
def search(style_font_path, num, top, bottom, trial):
    style_name = os.path.splitext(os.path.basename(style_font_path))[0]
    with open(f'../statistic/loss_dictionary/{style_name}_kanji.json', 'r') as f:
        all_loss_info = json.load(f)

    columns = list(all_loss_info.keys())
    index = list(kanji_chars)

    data = np.zeros((len(index), len(columns)))
    for i in range(len(columns)):
        tmp_column = [info['L1'] for info in list(all_loss_info[columns[i]].values())]
        data[:, i] = tmp_column

    result_info = {}
    indexes = [0] * num
    marked_set = set()
    for i in tqdm(range(trial)):
      while True:
        for i in range(num):
          indexes[i] = random.randint(0, len(index) - 1)
        sorted_indexed = sorted(indexes)
        if tuple(sorted_indexed) not in marked_set:
          marked_set.add(tuple(sorted_indexed))
          break
      result = {}
      chars = ''
      for i in range(num):
        chars += index[sorted_indexed[i]]
      tmp_data = data[:, sorted_indexed]
      loss_amount = tmp_data.min(axis=1).sum()
      loss_min_index = tmp_data.argmin(axis=1, )

      result['index'] = indexes
      result['loss_amount'] = loss_amount
      result['loss_min_index'] = [int(l) for l in loss_min_index]
      result_info[chars] = result
    
    sorted_result_info = {
      key: value for key, value in sorted(result_info.items(), key=lambda item: item[1]['loss_amount'])
    }
    top_bottom_result_info = {}
    top_keys = list(sorted_result_info.keys())[:top]
    bottom_keys = list(sorted_result_info.keys())[-bottom:]
    top_values = list(sorted_result_info.values())[:top]
    bottom_values = list(sorted_result_info.values())[-bottom:]
    for key, value in zip(top_keys, top_values):
        top_bottom_result_info[key] = value
    for key, value in zip(bottom_keys, bottom_values):
        top_bottom_result_info[key] = value

    result_dir_path = f'../statistic/loss_dictionary/{style_name}'
    result_file_path = os.path.join(result_dir_path, f'{num}_{trial}.json')
    if not os.path.exists(result_dir_path):
      os.makedirs(result_dir_path)
    with open(result_file_path, 'w') as f:
      json.dump(top_bottom_result_info, f)

In [ ]:
num = 4
top = 100
bottom = 100
trial = 1000000

style_font_paths = [
    'BIZUDPMincho-Regular.ttf',
    '851H-kktt_004.ttf',
    'JP_NotoSerifJP-Regular.otf',
    '03スマートフォントUI.otf',
    'Corporate-Logo-Rounded-Bold-ver3.otf',
    '851CHIKARA-DZUYOKU_kanaA_004.ttf',
    'JK-Maru-Gothic-M.otf',
]

for style_font_path in tqdm(style_font_paths):
    style_font_path = os.path.join('../fonts', style_font_path)
    search(style_font_path, num, top, bottom, trial)

  8%|▊         | 82538/1000000 [00:31<05:30, 2774.81it/s]


 16%|█▌        | 160443/1000000 [01:15<04:33, 3065.25it/s]


 24%|██▎       | 237193/1000000 [01:52<04:45, 2668.80it/s]


 31%|███▏      | 313112/1000000 [02:36<04:01, 2839.47it/s]


 39%|███▊      | 386629/1000000 [04:11<03:22, 3025.11it/s]


...

 75%|███████▍  | 746343/1000000 [07:50<01:22, 3078.14it/s]


 83%|████████▎ | 828316/1000000 [08:18<01:00, 2852.90it/s]


 91%|█████████ | 908432/1000000 [11:07<00:31, 2900.28it/s]


 99%|█████████▉| 990676/1000000 [11:34<00:03, 2938.42it/s]


100%|██████████| 1000000/1000000 [11:37<00:00, 1433.28it/s][A


In [27]:
style_name = 'JK-Maru-Gothic-M'
result_dir_path = f'../statistic/loss_dictionary/{style_name}'
result_file_path = os.path.join(result_dir_path, f'{num}_{trial}.json')
with open(result_file_path, 'r') as f:
    result_info = json.load(f)

In [28]:
result_info

{'鹿焦錬': {'index': [1981, 1428, 1862],
  'loss_amount': 364.5803938591853,
  'loss_min_index': [1,
   0,
   0,
   2,
   2,
   2,
   0,
   2,
   2,
   2,
   2,
   2,
   1,
   2,
   2,
   2,
   2,
   2,
   0,
   0,
   2,
   0,
   2,
   2,
   2,
   2,
   2,
   2,
   0,
   2,
   2,
   1,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   1,
   0,
   2,
   0,
   2,
   1,
   2,
   2,
   2,
   2,
   2,
   2,
   1,
   0,
   0,
   2,
   0,
   2,
   2,
   2,
   2,
   2,
   0,
   2,
   2,
   2,
   1,
   2,
   2,
   2,
   2,
   2,
   0,
   2,
   2,
   2,
   2,
   1,
   2,
   0,
   0,
   2,
   2,
   0,
   2,
   2,
   0,
   2,
   2,
   2,
   2,
   0,
   2,
   2,
   2,
   2,
   2,
   1,
   2,
   2,
   0,
   2,
   2,
   0,
   2,
   0,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   0,
   2,
   2,
   0,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   0,
   2,
   2,
   2,
   0,
   0,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   2,
   0,
   2,
   2,
   1,
   2,
   2,
   2,
   2,
   2,
   2,
  

In [11]:
result_info

{'更題中慮': {'index': [415, 2036, 772, 314],
  'loss_amount': 311.9430834464729,
  'loss_min_index': [2,
   2,
   0,
   3,
   1,
   1,
   3,
   3,
   1,
   1,
   1,
   1,
   2,
   2,
   2,
   3,
   0,
   0,
   2,
   3,
   1,
   3,
   1,
   3,
   1,
   3,
   1,
   1,
   1,
   0,
   1,
   1,
   2,
   1,
   3,
   1,
   1,
   1,
   2,
   2,
   1,
   1,
   1,
   3,
   1,
   1,
   3,
   1,
   1,
   1,
   1,
   1,
   2,
   3,
   2,
   1,
   0,
   2,
   0,
   2,
   3,
   3,
   1,
   3,
   1,
   1,
   3,
   3,
   3,
   2,
   3,
   1,
   2,
   3,
   1,
   1,
   0,
   1,
   2,
   2,
   2,
   0,
   0,
   1,
   1,
   1,
   1,
   3,
   2,
   2,
   1,
   1,
   1,
   1,
   0,
   1,
   0,
   3,
   1,
   2,
   3,
   1,
   1,
   1,
   3,
   3,
   2,
   3,
   3,
   1,
   1,
   0,
   1,
   0,
   1,
   1,
   0,
   3,
   2,
   2,
   2,
   0,
   1,
   2,
   3,
   2,
   3,
   1,
   3,
   1,
   3,
   3,
   3,
   2,
   1,
   2,
   2,
   3,
   2,
   2,
   2,
   3,
   1,
   0,
   2,
   1,
   2,
   1,
   1,
   1,
   2

In [19]:
trial = 1000000000
trial / ((len(kanji_chars) ** 4) / 12) * 100

0.05764693913544064

In [20]:
(len(kanji_chars) ** 4) / 12

1734697479168.0

In [34]:
trial = 10000000
trial / ((len(kanji_chars) ** 3) / 12) * 100

1.231338619933012